# Comparing Mephysto measurements to TPS DICOM dose exports

PyMedPhys contains within it tools to read Mephysto files as well as tools to extract profiles out of DICOM dose files. This example combines these two features together to compare beam models from both the TPS and the independent check software to measurement.

So that the exported doses can be directly compared the Mephysto profiles are normalised by absolute dose and output factors so that the profile units of both the DICOM files and the measurements both end up as Gy / 100 MU.

In [ ]:
# Makes it so any changes in pymedphys is automatically
# propagate into the notebook without needing a kernel reset.
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

In [ ]:
import re
import operator
import pathlib
import zipfile
import urllib.request

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pydicom

import pymedphys.dicom
import pymedphys.labs.tpscompare

## Download and extract the data

For this example we will be using a 350 MB zip file which contains a range of DICOM dose files as well as mephysto measurement files. We will now download these if the `data/tpscompare` directory doesn't already exist

In [ ]:
DATA_DIR = pathlib.Path('data')
DATA_DIR.mkdir(exist_ok=True)
ROOT_DIR = DATA_DIR.joinpath("tpscompare")

if not ROOT_DIR.exists():
    zip_filepath = DATA_DIR.joinpath('tpscompare.zip')
    
    url = 'https://zenodo.org/record/3351939/files/tpscompare.zip?download=1'
    urllib.request.urlretrieve(url, zip_filepath)
    
    with zipfile.ZipFile(zip_filepath, 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR)

## Defining the data directories

The data we just extracted is stored in the following way: 

![](./img/file-tree.png)

There are two locations where there are DICOM dose files, `Beam Models/DoseCHECK` and `Beam Models/Monaco`. There is also a plan file placed at `Beam Models/plan.dcm` which is the plan file that was used to create those doses.

Under `Measurements` are a range of Mephysto `.mcc` files, as well as three `.csv` files.

In [ ]:
BEAM_MODELS_DIR = ROOT_DIR.joinpath("Beam Models")

PLAN_FILEPATH = BEAM_MODELS_DIR.joinpath("plan.dcm")
MONACO_DICOM_DIR = BEAM_MODELS_DIR.joinpath("Monaco")
DOSECHECK_DICOM_DIR = BEAM_MODELS_DIR.joinpath("DoseCHECK")

MEASUREMENTS_DIR = ROOT_DIR.joinpath("Measurements")

RESULTS = ROOT_DIR.joinpath("Results")

## The dose normalisation definitions

To rescale the measurements to be `Gy / 100 MU` the absolute dose calibration, the total output factors, and any wedge transmission factors need to be supplied. In this example these are provided within `Calibration.csv`, `OutputFactors.csv` and `WedgeTransmissionFactors.csv`.

Here are the contents of those files:

In [ ]:
calibrated_doses_table = pd.read_csv(
    MEASUREMENTS_DIR.joinpath('Calibration.csv'), index_col=0)
calibrated_doses = calibrated_doses_table['d10 @ 90 SSD']

calibrated_doses_table

In [ ]:
output_factors = pd.read_csv(
    MEASUREMENTS_DIR.joinpath('OutputFactors.csv'), index_col=0)
output_factors

In [ ]:
wedge_transmission_table = pd.read_csv(
    MEASUREMENTS_DIR.joinpath('WedgeTransmissionFactors.csv'), index_col=0)
data_column_name = wedge_transmission_table.columns[0]
wedge_transmissions = wedge_transmission_table[data_column_name]

wedge_transmission_table

## Creating a list of beams to check

To know which beams we want to check we need to create a list. Since we have a list already in our `Beam Models` directories, lets use those to make a list

In [ ]:
keys = [
    path.stem
    for path in MONACO_DICOM_DIR.glob('*.dcm')
]

keys

## Defining the absolute doses at 100 mm depth for 90 SSD

For each profile and depth dose we want to compare we need to determine the absolute dose to normalise to. Here we are using the keys defined above to look up the `.csv` files loaded above to create a dictionary from each key to it's corresponding absolute dose at 100 mm depth @ 90 SSD.

In [ ]:
regex_string = r'(\d\dMV(FFF)?) (\d\dx\d\d) ((\bOpen\b)|(\bWedge\b))'

def get_energy_field_block(key):
    match = re.match(regex_string, key)
    return match.group(1), match.group(3), match.group(4)

In [ ]:
absolute_doses = {}

for key in keys:
    energy, field, block = get_energy_field_block(key)
        
    if block == 'Wedge':
        wtf = wedge_transmissions[energy]
    else:
        wtf = 1
        
    output_factor = output_factors[f'{field} {block}'][energy]
    calibrated_dose = calibrated_doses[energy]
    
    absolute_dose = calibrated_dose * output_factor * wtf
    absolute_doses[key] = absolute_dose
    

absolute_doses

## The `load_and_normalise_mephysto` function

PyMedPhys exports a function `load_and_normalise_mephysto`. The docstring for this function is printed below.

In [ ]:
print(pymedphys.labs.tpscompare.load_and_normalise_mephysto.__doc__)

### Using `load_and_normalise_mephysto`

To use this function we must pass a directory, a regex string, an the absolute dose dictionary provided above as well as the normalisation depth defined in mm. We do this below:

In [ ]:
absolute_scans_per_field = pymedphys.labs.tpscompare.load_and_normalise_mephysto(
    MEASUREMENTS_DIR, 
    r'(\d\dMV(FFF)? \d\dx\d\d ((\bOpen\b)|(\bWedge\b)))\.mcc', 
    absolute_doses, 100)

Of note is the regex string. To get an understanding of what this string is doing see https://regex101.com/r/DgC3MZ/1.

In [ ]:
mephysto_keys = list(absolute_scans_per_field.keys())
assert set(mephysto_keys) == set(keys), (
    f'Mephysto keys do not agree.\nMephysto Keys: f{mephysto_keys}\nOriginal Keys: f{keys}')

## Loading DICOM Files

Next we wish to load up our DICOM files. We will do this based upon the keys we have defined.

In [ ]:
def load_dicom_files(directory, keys):
    dicom_file_map = {
        key: directory.joinpath(f'{key}.dcm')
        for key in keys
    }
    
    dicom_dataset_map = {
        key: pydicom.read_file(str(dicom_file_map[key]), force=True)
        for key in keys
    }
    
    return dicom_dataset_map

In [ ]:
monaco_dicom_dataset_map = load_dicom_files(MONACO_DICOM_DIR, keys)
dosecheck_dicom_dataset_map = load_dicom_files(DOSECHECK_DICOM_DIR, keys)

In [ ]:
dicom_plan = pydicom.read_file(str(PLAN_FILEPATH), force=True)

## Plotting

### Creating the plotting functions

Here we create a set of functions which will be reused to display the overlayed doses and their dose differences.

In [ ]:
getter = operator.itemgetter('displacement', 'dose')

def plot_one_axis(ax, displacement, meas_dose, model_dose):
    diff = 100 * (model_dose - meas_dose) / meas_dose
    
    lines = []
    
    lines += ax.plot(displacement, meas_dose, label='Measured Dose')
    lines += ax.plot(displacement, model_dose, label='Model Dose')
    ax.set_ylabel('Dose (Gy / 100 MU)')
    
    x_bounds = [np.min(displacement), np.max(displacement)]
    ax.set_xlim(x_bounds)

    ax_twin = ax.twinx()

    lines += ax_twin.plot(
        displacement, diff, color='C3', lw=0.5,
        label=r'% Residuals [100 $\times$ (Model - Meas) / Meas]')
    ax_twin.plot(x_bounds, [0, 0], '--', color='C3', lw=0.5)
    ax_twin.set_ylabel(r'% Dose difference [100 $\times$ (Model - Meas) / Meas]')

    labels = [l.get_label() for l in lines]
    
    ax.legend(lines, labels, loc='lower left')
    
    return ax_twin



def plot_tps_meas_diff(displacement, meas_dose, monaco_dose, dosecheck_dose):
    fig, ax = plt.subplots(1, 2, figsize=(10.5,6), sharey=True)
    ax[1].yaxis.set_tick_params(which='both', labelbottom=True)

    ax_twin = list()
    
    ax_twin.append(plot_one_axis(ax[0], displacement, meas_dose, monaco_dose))
    ax_twin.append(plot_one_axis(ax[1], displacement, meas_dose, dosecheck_dose))
    
    ax_twin[1].get_shared_y_axes().join(ax_twin[1], ax_twin[0])
    ax_twin[1].set_ylim([-5, 5])
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.4, top=0.86)
    
    return fig, ax

### Creating the depth dose plotting function

Here we create the `plot_depth_dose_diff` function.

Of note it can be seen that there is a `/ 10` at the end of each dose loading section. This is because 1000 MU was used per beam to generate the doses in the dose files. By diving by 10 this scales these doses to `Gy / 100 MU`.

In [ ]:
def plot_depth_dose_diff(key, absolute_scans_per_field, 
                         monaco_dicom_dataset_map, dosecheck_dicom_dataset_map, 
                         dicom_plan):
    depth, meas_dose = getter(absolute_scans_per_field[key]['depth_dose'])
    
    monaco_dose = pymedphys.dicom.depth_dose(
        depth, monaco_dicom_dataset_map[key], dicom_plan) / 10
    dosecheck_dose = pymedphys.dicom.depth_dose(
        depth, dosecheck_dicom_dataset_map[key], dicom_plan) / 10

    fig, ax = plot_tps_meas_diff(depth, meas_dose, monaco_dose, dosecheck_dose)
    fig.suptitle(f'Depth Dose Comparisons | {key}', fontsize="x-large")
    ax[0].set_title("Monaco")
    ax[1].set_title("DoseCHECK")
    
    ax[0].set_xlabel("Depth (mm)")
    ax[1].set_xlabel("Depth (mm)")

In the above function `pymedphys.dicom.depth_dose` was used. See below for its docstring.

In [ ]:
print(pymedphys.dicom.depth_dose.__doc__)

### Plotting the depth doses

Now we have defined our functions, let's step through each of our keys, plot and save the results as png images in the results directory.

In [ ]:
for key in keys:
    plot_depth_dose_diff(key, absolute_scans_per_field, 
                         monaco_dicom_dataset_map, dosecheck_dicom_dataset_map, 
                         dicom_plan)
    filename = RESULTS.joinpath(f'{key}_pdd.png')
    plt.savefig(filename)
    plt.show()

### Creating the profile plotting function

Here we create the `plot_profile_diff` function.


In [ ]:
def plot_profile_diff(key, depth, direction, absolute_scans_per_field, 
                      monaco_dicom_dataset_map, dosecheck_dicom_dataset_map, 
                      dicom_plan):
    displacement, meas_dose = getter(
        absolute_scans_per_field[key]['profiles'][depth][direction])
    
    monaco_dose = pymedphys.dicom.profile(
        displacement, depth, direction, 
        monaco_dicom_dataset_map[key], dicom_plan) / 10
    dosecheck_dose = pymedphys.dicom.profile(
        displacement, depth, direction, 
        dosecheck_dicom_dataset_map[key], dicom_plan) / 10

    fig, ax = plot_tps_meas_diff(displacement, meas_dose, monaco_dose, dosecheck_dose)
    fig.suptitle(
        f'{direction.capitalize()} Profile Comparisons | {key} | Depth: {depth} mm',
        fontsize="x-large")
    ax[0].set_title("Monaco")
    ax[1].set_title("DoseCHECK")
    
    ax[0].set_xlabel("Displacement from CRA (mm)")
    ax[1].set_xlabel("Displacement from CRA (mm)")

In the above function `pymedphys.dicom.profile` was used. See below for its docstring.

In [ ]:
print(pymedphys.dicom.profile.__doc__)

### Plotting the profiles

For the profiles we need to both step through our keys, depths, and our directions. Once we have done that, plot the profile and save the result with an appropriately named filename.

In [ ]:
for key in keys:
    depths = absolute_scans_per_field[key]['profiles'].keys()
    for depth in depths:
        for direction in ['inplane', 'crossplane']:
            plot_profile_diff(key, depth, direction, absolute_scans_per_field, 
                              monaco_dicom_dataset_map, dosecheck_dicom_dataset_map, 
                              dicom_plan)
            filename = RESULTS.joinpath(f'{key}_profile_{depth}mm_{direction}.png')
            plt.savefig(filename)
            plt.show()